recommendation sys


In [14]:
import numpy as np
import pandas as pd

In [15]:
brand_data = pd.read_csv("brand_dataset.csv")
influencer_data = pd.read_csv("influencer_dataset.csv")

In [16]:
brand_data.head()

,brand_id,brand_name,product_category,audience_age,audience_gender,audience_location
0,1,Brand72,Nutrition Products,15-24,Female,Kerala
1,2,Brand88,Smartphones,All,Male,Gujarat
2,3,Brand55,Home Appliances,18-30,Male,Kerala
3,4,Brand84,Gadgets,15-24,Male,Kerala
4,5,Brand43,Home Appliances,40-50,All,Tamil Nadu


In [17]:
brand_data['brand_name'].unique

<bound method Series.unique of 0      Brand72
1      Brand88
2      Brand55
3      Brand84
4      Brand43
        ...   
595     Brand8
596    Brand80
597    Brand29
598     Brand7
599    Brand89
Name: brand_name, Length: 600, dtype: object>

In [19]:
influencer_data.head()

,Influencer_ID,Influencer_name,Handle,Followers,Posts,Likes,Category,Sentiment,Location,Age,Gender,Platform,Engagement_rate
0,1,Jane Doe,@emma.t,340770,986,31587,Food,Positive,Kerala,29,Male,TikTok,9.269302
1,2,Mia Lee,@jane.doe,702603,847,49725,Fashion,Positive,Gujarat,31,All,Twitter,7.077254
2,3,John Smith,@noah.d,113855,241,41587,Food,Positive,Kerala,25,Male,Instagram,36.526283
3,4,Emma Taylor,@jane.doe,202147,944,8042,Lifestyle,Negative,Kerala,49,All,Facebook,3.978293
4,5,Mason White,@john.smith,495171,508,30531,Lifestyle,Negative,Tamil Nadu,19,Female,Instagram,6.165749


In [15]:
brand_data.isnull().sum()

brand_id             0
brand_name           0
product_category     0
audience_age         0
audience_gender      0
audience_location    0
dtype: int64

In [14]:

influencer_data.isnull().sum()

Influencer_ID      0
Influencer Name    0
Handle             0
Followers          0
Posts              0
Likes              0
Category           0
Sentiment          0
Location           0
Engagement Rate    0
dtype: int64